# Generación de instancias 

En este notebook generaremos los sets de instancias que usaremos para experimentar.

- Estas instancias van a ser guardadas en la carpeta __instancias__.

In [1]:
import os
import shutil
import random, math, subprocess
import pandas as pd

Funciones útiles para guardar las instancias:

In [2]:
# Creación de la carpeta instancias

def crear_carpeta(dir):
    if os.path.exists(dir) : shutil.rmtree(dir)
    os.makedirs(dir)

In [3]:
# Guardado de la instancia

def save_instance(dataset, instance_name, r_i, r_e, m_p_1, n, iso, t_i, t_e, ninst):
    with open(F"instancias/{dataset}/{instance_name}.in", "w") as f:
        print(r_i, r_e, m_p_1, n, iso, ninst, file=f)
        for i in range(0, ninst): 
            for j in range(0, n):
                print(t_i[i][j], end=" ", file=f)
            for j in range(0, n):
                print(t_e[i][j], end=" ", file=f)
            print(end = "\n", file=f)

## Dataset 1 : Pared exterior de alta temperatura

- Este primer dataset contempla una discretización del horno con temperaturas fijas $T_i = 1500~$ y $~T_e = 200$.
- $n = 40$
- $m \in \{x~:~x = i\cdot 2, ~ i = 1\ldots20 \land x \geq 4\}$
- $ninst = 1$, con todas las instancias que surgen de los pares $(m,n)$
- La isoterma buscada será 500
- $r_i = 10$
- $r_e = 15$

In [5]:
crear_carpeta('instancias/D1')

index_filas = []
T_i = 1500
T_e = 200
r_i = 10
r_e = 15
iso = 500
ninst = 1
n = 40
m = 40

for j in range(4,m+1,2):
    t_i = [[T_i for i in range(n)]]
    t_e = [[T_e for i in range(n)]]
    save_instance("D1", F"D1_{j}_{n}", r_i, r_e, j, n, iso, t_i, t_e, ninst)
    index_filas.append(["D1", F"D1_{j}_{n}", r_i, r_e, j, n, iso, ninst,F"instancias/D1/D1_{j}_{n}.in"])

pd.DataFrame(index_filas, columns=["Dataset", "Nombre de instancia", "Radio interno", "Radio externo", "m+1", "n", "Isoterma", "Número de instancia", "Archivo"]).to_csv("instancias/D1/indice.csv", index=False, header=True, sep=',')

## Dataset 2 : Temperaturas que cambian con el tiempo

- Este segundo dataset contempla una discretización del horno con temperaturas que varían en el tiempo, siendo estos cambios representados instancia a instancia con una cantidad de ángulos y radios fija. Los cambios de temperatura repetarán una distribución normal, intentando simular el posible error de medición de un sensor en el horno, con $\sigma = \frac{\mu}{50}$.
- $n = 40$
- $m = 30$
- $ninst = 30$
- La isoterma buscada será 500
- $r_i = 15$
- $r_e = 5$
- $T_i \sim N(1500, 30)$
- $T_e \sim N(200, 4)$

In [6]:
crear_carpeta('instancias/D2')

index_filas = []
n = 40
m = 30
r_i = 10
r_e = 15
iso = 500
ninst = 30
T_i = 1500
T_e = 200
t_i = []
t_e = []

for h in range(ninst):
    t_i.append([random.normalvariate(T_i, T_i//50) for i in range(n)])
    t_e.append([random.normalvariate(T_e, T_e//50) for i in range(n)])
    index_filas.append(["D2", F"D2_{m}_{n}", r_i, r_e, m, n, iso, h+1,F"instancias/D2/D2_{m}_{n}.in"])
    
save_instance("D2", F"D2_{m}_{n}",r_i,r_e,m,n,iso,t_i,t_e,ninst)
pd.DataFrame(index_filas, columns=["Dataset", "Nombre de instancia", "Radio interno", "Radio externo", "m+1", "n", "Isoterma", "Número de instancia", "Archivo"]).to_csv("instancias/D2/indice.csv", index=False, header=True, sep=',')

## Dataset 3 : Horno con sistema de refrigeración no funcional

- Este tercer dataset contempla una discretización del horno parecida al dataset previo, con temperaturas fijas $T_i = 50~$ y $~T_e = 1500$. La distinción está en que itnenta simular uno de los potenciales escenarios de peligro al momento de utilizar un horno de esta indole, donde parte del sistema de refrigeración del mismo no funciona como corresponde, permitiendo que determina sección del horno comience a recalentarse.
- $n = 40$
- $m = 30$
- $ninst = 50$
- La isoterma buscada será 500.
- $r_i = 15$
- $r_e = 5$
- $T_i = 1500$
- $T_e = 200$
- $k = 10$, este $k$ es el incremento de temperatura que sufre la mitad del horno en cada instancia.

In [7]:
crear_carpeta('instancias/D3')

index_filas = []
n = 40
m = 30
r_i = 10
r_e = 15
iso = 500
ninst = 50
T_i = 1500
T_e = 200
t_i = []
t_e = []

for h in range(ninst):
    t_i.append([T_i for i in range(n//2)] + [T_i + 10*h for i in range(n//2)])
    t_e.append([T_e for i in range(n//2)] + [T_e + 10*h for i in range(n//2)])
    index_filas.append(["D3", F"D3_{m}_{n}", r_i, r_e, m, n, iso, h,F"instancias/D3/D3_{m}_{n}.in"])
    
save_instance("D3", F"D3_{m}_{n}",r_i,r_e,m,n,iso,t_i,t_e,ninst)
pd.DataFrame(index_filas, columns=["Dataset", "Nombre de instancia", "Radio interno", "Radio externo", "m+1", "n", "Isoterma", "Número de instancia", "Archivo"]).to_csv("instancias/D3/indice.csv", index=False, header=True, sep=',')

## Dataset 4 : Horno con sensores fallados

- Este tercer dataset contempla una discretización del horno parecida al dataset previo, con temperaturas fijas $T_i = 50~$ y $~T_e = 1500$. En este dataset habrá sensores que no están configurados de forma adecuada y miden mal la temperatura a través del tiempo. Los cambios de temperatura repetarán una distribución normal, intentando simular el posible error de medición de un sensor en el horno, con $\sigma = \frac{\mu}{50}$, mientras que los sensores defectuosos con $\sigma = \frac{\mu}{5}$.
- $n = 40$
- $m = 30$
- $ninst = 50$
- La isoterma buscada será 1000.
- $r_i = 15$
- $r_e = 5$
- $T_i \sim N(1500, 30)$
- $T_e \sim N(500, 4)$

In [8]:
crear_carpeta('instancias/D4')

index_filas = []
n = 40
m = 30
r_i = 10
r_e = 15
iso = 1000
ninst = 30
T_i = 1500
T_e = 500
t_i = []
t_e = []
s_defectuosos =  [random.randint(1,n-1) for i in range(5)]

for h in range(ninst):
    t_i.append([random.normalvariate(T_i, T_i//50) for i in range(n)])
    t_e.append([random.normalvariate(T_e, T_e//50) for i in range(n)])
    for j in s_defectuosos:
        t_e[h][j] = random.normalvariate(T_e, T_e//5)
    index_filas.append(["D4", F"D4_{m}_{n}", r_i, r_e, m, n, iso, h+1,F"instancias/D4/D4_{m}_{n}.in"])
    
save_instance("D4", F"D4_{m}_{n}",r_i,r_e,m,n,iso,t_i,t_e,ninst)
pd.DataFrame(index_filas, columns=["Dataset", "Nombre de instancia", "Radio interno", "Radio externo", "m+1", "n", "Isoterma", "Número de instancia", "Archivo"]).to_csv("instancias/D4/indice.csv", index=False, header=True, sep=',')